In [3]:
!pip install pymilvus

  Using cached pymilvus-2.6.9-py3-none-any.whl.metadata (6.8 kB)
  Using cached orjson-3.11.7-cp313-cp313-win_amd64.whl.metadata (43 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pymilvus-2.6.9-py3-none-any.whl (301 kB)
   ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
   ------------------------------------- -- 4.5/4.8 MB 26.3 MB/s eta 0:00:01
   ---------------------------------------- 4.8/4.8 MB 23.1 MB/s  0:00:00
Using cached orjson-3.11.7-cp313-cp313-win_amd64.whl (124 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   -------------------------------- ------- 7.9/9.7 MB 38.0 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 35.3 MB/s  0:00:00
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   --------------------------- ------------ 8.4/12.3 MB 40.7 MB/s eta 0:00:01
   ---------------------------------------- 12.3/12.3 MB 35.0 MB/s  0:00:00
   ------------


[notice] A new release of pip is available: 25.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from pymilvus import connections, utility

# Connect to Milvus
connections.connect(
    alias="default",
    host="localhost",
    port="19530"
)

print("✅ Connected to Milvus")

# Optional: Check existing collections
collections = utility.list_collections()
print("Existing collections:", collections)


✅ Connected to Milvus
Existing collections: []


In [5]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),

    FieldSchema(name="contract_id", dtype=DataType.INT64),

    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1024),

    FieldSchema(name="clause_type", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="contract_type", dtype=DataType.VARCHAR, max_length=100),
    FieldSchema(name="text_chunk", dtype=DataType.VARCHAR, max_length=5000),
]



In [6]:
schema = CollectionSchema(fields, description="Policy Clause Embeddings")

collection = Collection(
    name="legal_policy_vectors",
    schema=schema
)

index_params = {
    "metric_type": "COSINE",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 128}
}

collection.create_index(
    field_name="embedding",
    index_params=index_params
)

collection.load()

In [8]:
from sentence_transformers import SentenceTransformer

# Load BGE-M3
model = SentenceTransformer("BAAI/bge-m3")

# Confirm dimension
test_embedding = model.encode("test sentence", normalize_embeddings=True)
print("Embedding dimension:", len(test_embedding))

c:\Users\mbalasubramanian\Documents\rag_bot_assignment\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\mbalasubramanian\Documents\rag_bot_assignment\venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mbalasubramanian\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Pyth

Embedding dimension: 1024


In [9]:
def chunk_text(text, chunk_size=500):
    chunks = []
    current_chunk = ""

    for sentence in text.split(". "):
        if len(current_chunk) + len(sentence) <= chunk_size:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

In [ ]:
import os

def insert_document(file_path, clause_type, contract_type):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    chunks = chunk_text(text)

    # IMPORTANT: normalize for cosine similarity
    embeddings = model.encode(
        chunks,
        normalize_embeddings=True
    )

    data = [
        embeddings.tolist(),
        [os.path.basename(file_path)] * len(chunks),
        [clause_type] * len(chunks),
        [contract_type] * len(chunks),
        chunks
    ]

    collection.insert(data)
    collection.flush()

    print(f"✅ Inserted {len(chunks)} chunks from {file_path}")

In [10]:
from pymilvus import utility

utility.drop_collection("legal_policy_vectors")

SchemaNotReadyException: <SchemaNotReadyException: (code=1, message=Collection 'legal_policy_vectors' not exist, or you can pass in schema to create one.)>